<a href="https://colab.research.google.com/github/tsuli/LSC_project/blob/main/Task2_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# specify the data file name and url
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/'
datafile = url + 'UCI%20HAR%20Dataset.zip'

# download the zip file from the web server using curl
!curl $datafile --output UCI_HAR_Dataset.zip

# unzip the file
!unzip -qq UCI_HAR_Dataset.zip

# change the directory name to remove spaces
!mv -f UCI\ HAR\ Dataset UCI_HAR_DATASET

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.1M  100 58.1M    0     0  31.9M      0  0:00:01  0:00:01 --:--:-- 31.9M


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
# load the features and labels (subtract 1 as the labels aren't indexed from 0)
ytest = np.loadtxt('UCI_HAR_DATASET/test/y_test.txt').astype(np.float32) -1
ytrain = np.loadtxt('UCI_HAR_DATASET/train/y_train.txt').astype(np.float32) -1

# load the x,y,z body accelerations test data
xx=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_x_test.txt').astype(np.float32) 
yy=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_y_test.txt').astype(np.float32) 
zz=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_z_test.txt').astype(np.float32) 

# concatenate the arrays along the last dimension
xtest = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)
# (using None here adds an extra dimension of size 1 to the end of the array)

# follow the same approach for the train data
xx=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_x_train.txt').astype(np.float32) 
yy=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_y_train.txt').astype(np.float32) 
zz=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_z_train.txt').astype(np.float32) 

xtrain = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)

xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)
ytrain_enc = tf.keras.utils.to_categorical(ytrain)
yvalid_enc = tf.keras.utils.to_categorical(yvalid)
ytest_enc = tf.keras.utils.to_categorical(ytest)
xtrain = xtrain.astype(np.float32)
xvalid = xvalid.astype(np.float32)
xtest = xtest.astype(np.float32)

print(xtrain.shape, ytrain_enc.shape, xvalid.shape, yvalid_enc.shape, xtest.shape, ytest_enc.shape)

(5881, 128, 3) (5881, 6) (1471, 128, 3) (1471, 6) (2947, 128, 3) (2947, 6)


In [ ]:
input_layer = tf.keras.Input(shape=(128,3))
output_layer1 = tf.keras.layers.Conv1D(filters=96, kernel_size=(16), padding = 'same')(input_layer)
output_layer2 = tf.keras.layers.BatchNormalization()(output_layer1)
output_layer3 = tf.keras.layers.ReLU()(output_layer2)
output_layer4 = tf.keras.layers.GlobalAveragePooling1D()(output_layer3)
output_layer5 = tf.keras.layers.Dense(units=6,activation=tf.nn.softmax)(output_layer4)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer5)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]

In [ ]:
model.fit(xtrain,  ytrain_enc, 
          epochs=500,
          batch_size=256, 
          callbacks=callbacks,
          validation_data=(xvalid, yvalid_enc))

Epoch 1/500
23/23 [==============================] - 3s 91ms/step - loss: 1.9548 - accuracy: 0.1741 - val_loss: 1.7917 - val_accuracy: 0.2162
Epoch 2/500
23/23 [==============================] - 2s 81ms/step - loss: 1.8151 - accuracy: 0.2236 - val_loss: 1.7833 - val_accuracy: 0.3807
Epoch 3/500
23/23 [==============================] - 2s 78ms/step - loss: 1.7052 - accuracy: 0.3124 - val_loss: 1.7740 - val_accuracy: 0.5343
Epoch 4/500
23/23 [==============================] - 2s 78ms/step - loss: 1.6186 - accuracy: 0.3804 - val_loss: 1.7639 - val_accuracy: 0.5914
Epoch 5/500
23/23 [==============================] - 2s 78ms/step - loss: 1.5506 - accuracy: 0.4719 - val_loss: 1.7524 - val_accuracy: 0.5935
Epoch 6/500
23/23 [==============================] - 2s 80ms/step - loss: 1.4956 - accuracy: 0.5640 - val_loss: 1.7398 - val_accuracy: 0.5948
Epoch 7/500
23/23 [==============================] - 2s 78ms/step - loss: 1.4502 - accuracy: 0.6002 - val_loss: 1.7256 - val_accuracy: 0.5989
Epoch 

**Experiments with batch size** **(filter = 32, kernel_size = (4), lr=0.001)** 

**1,** s 4ms/step - loss: 0.5741 - accuracy: 0.7747 - val_loss: 0.
5855 - val_accuracy: 0.7485, epochs=100, batch_size=256






**2,** s 10ms/step - loss: 0.6661 - accuracy: 0.7368 - val_loss: 0.6702 - val_accuracy: 0.6621, epochs=100, batch_size=64

**3,** 1s 29ms/step - loss: 0.7814 - accuracy: 0.7021 - val_loss: 0.7837 - val_accuracy: 0.6234, epochs=30, batch_size=256

**4,** 30ms/step - loss: 0.7331 - accuracy: 0.7021 - val_loss: 0.7508 - val_accuracy: 0.6166, epochs=200, batch_size=256

**5,** 1s 57ms/step - loss: 0.7029 - accuracy: 0.7526 - val_loss: 0.7132 - val_accuracy: 0.6424, epochs=200, batch_size=512

**6,** 1s 32ms/step - loss: 0.7416 - accuracy: 0.7060 - val_loss: 0.7461 - val_accuracy: 0.6506, epochs=200, batch_size=320

**7,** 1s 23ms/step - loss: 0.6549 - accuracy: 0.7393 - val_loss: 0.6881 - val_accuracy: 0.6315, epochs=200, batch_size=192

**8,** 34ms/step - loss: 0.6833 - accuracy: 0.7487 - val_loss: 0.6921 - val_accuracy: 0.6254, epochs=200, batch_size=256

**Experiments with other parameters**

In [ ]:
filter = 32, kernel_size = (4),
lr=0.1,
epoch 100 (stopped at 5),
batch_size=256
1s 27ms/step - loss: 0.6199 - accuracy: 0.7060 - val_loss: 2.3839 - val_accuracy: 0.4657

In [ ]:
filter = 32, kernel_size = (4),
lr=0.0001,
epoch 500 (stopped at 500),
batch_size=256,
1s 33ms/step - loss: 0.6758 - accuracy: 0.7463 - val_loss: 0.6755 - val_accuracy: 0.7559

In [ ]:
filter = 96, kernel_size = (4),
lr=0.0001,
epochs=500 (stopped at 421), 
batch_size=256,
2s 80ms/step - loss: 0.6031 - accuracy: 0.7902 - val_loss: 0.6067 - val_accuracy: 0.8083

In [ ]:
filters=96, kernel_size=(16),
lr=0.0001,
epochs=500 (stopped below 500), 
batch_size=256,
86ms/step - loss: 0.4763 - accuracy: 0.8308 - val_loss: 0.4868 - val_accuracy: 0.8566